In [ ]:
import json

# Define the 'Source' stage of the pipeline
source_stage = {
  'Name': 'Source',
  'Actions': [
    {
      'Name': 'Source',
      'ActionTypeId': {
        'Category': 'Source',
        'Owner': 'ThirdParty',
        'Provider': 'GitHub',
        'Version': '1'
      },
      'OutputArtifacts': [
        {'Name': 'SourceOutput'}
      ],
      'Configuration': {
        'Owner': {'Ref': 'GitHubOwner'},
        'Repo': {'Ref': 'GitHubRepo'},
        'Branch': {'Ref': 'GitHubBranch'},
        'OAuthToken': {'Ref': 'GitHubToken'},
        'PollForSourceChanges': 'false'
      },
      'RunOrder': 1
    }
  ]
}

# Write the 'Source' stage to a JSON file
with open('source_stage.json', 'w') as f:
  json.dump(source_stage, f, indent=2)

In [ ]:
# Define the 'Build' stage of the pipeline
build_stage = {
  'Name': 'Build',
  'Actions': [
    {
      'Name': 'Build',
      'ActionTypeId': {
        'Category': 'Build',
        'Owner': 'AWS',
        'Provider': 'CodeBuild',
        'Version': '1'
      },
      'InputArtifacts': [
        {'Name': 'SourceOutput'}
      ],
      'OutputArtifacts': [
        {'Name': 'BuildOutput'}
      ],
      'Configuration': {
        'ProjectName': {'Ref': 'CodeBuildProjectName'}
      },
      'RunOrder': 1
    }
  ]
}

# Write the 'Build' stage to a JSON file
with open('build_stage.json', 'w') as f:
  json.dump(build_stage, f, indent=2)

In [ ]:
# Define the 'Deploy' stage of the pipeline
deploy_stage = {
  'Name': 'Deploy',
  'Actions': [
    {
      'Name': 'Deploy',
      'ActionTypeId': {
        'Category': 'Invoke',
        'Owner': 'AWS',
        'Provider': 'Lambda',
        'Version': '1'
      },
      'InputArtifacts': [
        {'Name': 'BuildOutput'}
      ],
      'Configuration': {
        'FunctionName': {'Ref': 'LambdaFunctionName'},
        'UserParameters': {'Fn::Sub': '{"BucketName":"${BucketName}"}'}
      },
      'RunOrder': 1
    }
  ]
}

# Write the 'Deploy' stage to a JSON file
with open('deploy_stage.json', 'w') as f:
  json.dump(deploy_stage, f, indent=2)

In [ ]:
# Load the stages from their respective JSON files
with open('source_stage.json', 'r') as f:
  source_stage = json.load(f)
with open('build_stage.json', 'r') as f:
  build_stage = json.load(f)
with open('deploy_stage.json', 'r') as f:
  deploy_stage = json.load(f)

# Combine the stages into a pipeline
pipeline = {
  'Type': 'AWS::CodePipeline::Pipeline',
  'Properties': {
    'Name': {'Ref': 'PipelineName'},
    'RoleArn': {'Fn::GetAtt': ['GithubBackupPipelineRole', 'Arn']},
    'Stages': [source_stage, build_stage, deploy_stage]
  }
}

# Write the pipeline to a JSON file
with open('pipeline.json', 'w') as f:
  json.dump(pipeline, f, indent=2)

In [ ]:
# Load the pipeline from its JSON file
with open('pipeline.json', 'r') as f:
  pipeline = json.load(f)

# Define the other CloudFormation resources
other_resources = {
  'GithubBackupPipelineRole': {
    'Type': 'AWS::IAM::Role',
    'Properties': {
      'AssumeRolePolicyDocument': {
        'Version': '2012-10-17',
        'Statement': [
          {
            'Effect': 'Allow',
            'Principal': {
              'Service': [
                'codepipeline.amazonaws.com'
              ]
            },
            'Action': 'sts:AssumeRole'
          }
        ]
      },
      'ManagedPolicyArns': [
        'arn:aws:iam::aws:policy/AmazonS3FullAccess',
        'arn:aws:iam::aws:policy/AWSCodeBuildAdminAccess',
        'arn:aws:iam::aws:policy/AWSLambda_FullAccess'
      ]
    }
  }
}

# Combine the pipeline with the other resources
template = {
  'Resources': {**other_resources, **pipeline}
}

# Write the template to a JSON file
with open('template.json', 'w') as f:
  json.dump(template, f, indent=2)

In [ ]:
import yaml

# Load the template from its JSON file
with open('template.json', 'r') as f:
  template = json.load(f)

# Convert the template to YAML
template_yaml = yaml.dump(template)

# Write the template to a YAML file
with open('template.yaml', 'w') as f:
  f.write(template_yaml)

In [ ]:
# Define the CloudFormation parameters
parameters = {
  'GitHubToken': {
    'Type': 'AWS::SSM::Parameter::Value<String>',
    'Default': '/api/github',
    'Description': 'The GitHub personal access token stored as a secure string in SSM Parameter Store.'
  }
}

# Add the parameters to the template
template['Parameters'] = parameters

# Write the updated template to a JSON file
with open('template.json', 'w') as f:
  json.dump(template, f, indent=2)

# Convert the updated template to YAML
template_yaml = yaml.dump(template)

# Write the updated template to a YAML file
with open('template.yaml', 'w') as f:
  f.write(template_yaml)

In [ ]:
!pip install PyGithub

In [ ]:
from github import Github
import os

# First create a Github instance:
# using an access token
g = Github(os.getenv('GITHUB_TOKEN'))

# Then play with your Github objects:
repo = g.get_repo('matthewhand/github-backup')

# Read the contents of the 'template.yaml' file
with open('template.yaml', 'r') as f:
  template_yaml = f.read()

# Read the contents of the 'buildspec.yml' file
with open('buildspec.yml', 'r') as f:
  buildspec_yml = f.read()

# Push the 'template.yaml' file to the repository
repo.create_file('template.yaml', 'Add template.yaml', template_yaml)

# Push the 'buildspec.yml' file to the repository
repo.create_file('buildspec.yml', 'Add buildspec.yml', buildspec_yml)

In [ ]:
# Define the contents of the 'buildspec.yml' file
buildspec_yml = '''
version: 0.2

phases:
  install:
    commands:
      - echo Checking for requirements.txt...
      - if [ -f requirements.txt ]; then echo requirements.txt found, installing requirements... && pip install -r requirements.txt; else echo requirements.txt not found, skipping...; fi
  build:
    commands:
      - echo Checking for template.yaml...
      - if [ -f template.yaml ]; then echo template.yaml found, running cfn-lint... && cfn-lint template.yaml; else echo template.yaml not found, skipping...; fi
'''

# Write the 'buildspec.yml' file
with open('buildspec.yml', 'w') as f:
  f.write(buildspec_yml)

In [ ]:
# Read the contents of the 'buildspec.yml' file again
with open('buildspec.yml', 'r') as f:
  buildspec_yml = f.read()

# Push the 'template.yaml' file to the repository
repo.create_file('template.yaml', 'Add template.yaml', template_yaml)

# Push the 'buildspec.yml' file to the repository
repo.create_file('buildspec.yml', 'Add buildspec.yml', buildspec_yml)

In [ ]:
# Define the contents of the 'README.md' file
readme_md = '''
# AWS IaC CI/CD Project

## Donation Button

If you find this project useful, you can support its development by making a donation. Think of it as buying me a coffee. Click on the button below to make a donation. You can choose an amount between 1 and 99.

[![Donate with Stripe](https://img.shields.io/badge/Donate%20with-Stripe-blue.svg)](https://buy.stripe.com/00g14peASeEd7xCcMM)

<img src="https://github.com/matthewhand/stripe-payment/raw/main/qr_00g14peASeEd7xCcMM.png" width="150" />

## TODO

- Macie Scan

## Notebooks

The files in the 'notebooks' folder are the raw files created by ChatGPT. They contain the Python code that was executed to develop this project.

![Human AI Conflict](https://github.com/matthewhand/stripe-payment/raw/main/human_ai_conflict.png)

The image above represents the symbiotic relationship between human and machine.

*"Imagination is not limited by what is, but rather, inspired by what could be." - Unknown
'''

# Write the 'README.md' file
with open('README.md', 'w') as f:
  f.write(readme_md)

In [ ]:
# Read the contents of the 'README.md' file
with open('README.md', 'r') as f:
  readme_md = f.read()

# Push the 'README.md' file to the repository
repo.create_file('README.md', 'Add README.md', readme_md)

In [ ]:
import boto3

# Create a session using your AWS credentials
session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_REGION')
)

# Create a CloudFormation client
cf = session.client('cloudformation')

# Read the contents of the 'template.yaml' file
with open('template.yaml', 'r') as f:
  template_yaml = f.read()

# Deploy the CloudFormation template
response = cf.create_stack(
    StackName='GithubBackupPipeline',
    TemplateBody=template_yaml,
    Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  response = cf.describe_stacks(StackName=stack_id)
  stack_status = response['Stacks'][0]['StackStatus']
  print(f'Stack status: {stack_status}')
  if stack_status.endswith('_COMPLETE') or stack_status.endswith('_FAILED'):
    break
  time.sleep(5)

In [ ]:
# Correct the type of the 'GitHubToken' parameter
template['Parameters']['GitHubToken']['Type'] = 'AWS::SSM::Parameter::Value<SecureString>'

# Write the corrected template to a JSON file
with open('template.json', 'w') as f:
  json.dump(template, f, indent=2)

# Convert the corrected template to YAML
template_yaml = yaml.dump(template)

# Write the corrected template to a YAML file
with open('template.yaml', 'w') as f:
  f.write(template_yaml)

In [ ]:
# Read the contents of the corrected 'template.yaml' file
with open('template.yaml', 'r') as f:
  template_yaml = f.read()

# Deploy the corrected CloudFormation template
response = cf.create_stack(
    StackName='GithubBackupPipeline',
    TemplateBody=template_yaml,
    Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  response = cf.describe_stacks(StackName=stack_id)
  stack_status = response['Stacks'][0]['StackStatus']
  print(f'Stack status: {stack_status}')
  if stack_status.endswith('_COMPLETE') or stack_status.endswith('_FAILED'):
    break
  time.sleep(5)

In [ ]:
# Correct the type of the 'GitHubToken' parameter
template['Parameters']['GitHubToken']['Type'] = 'AWS::SSM::Parameter::Value<String>'

# Write the corrected template to a JSON file
with open('template.json', 'w') as f:
  json.dump(template, f, indent=2)

# Convert the corrected template to YAML
template_yaml = yaml.dump(template)

# Write the corrected template to a YAML file
with open('template.yaml', 'w') as f:
  f.write(template_yaml)

In [ ]:
# Read the contents of the corrected 'template.yaml' file
with open('template.yaml', 'r') as f:
  template_yaml = f.read()

# Deploy the corrected CloudFormation template
response = cf.create_stack(
    StackName='GithubBackupPipeline',
    TemplateBody=template_yaml,
    Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  response = cf.describe_stacks(StackName=stack_id)
  stack_status = response['Stacks'][0]['StackStatus']
  print(f'Stack status: {stack_status}')
  if stack_status.endswith('_COMPLETE') or stack_status.endswith('_FAILED'):
    break
  time.sleep(5)

In [ ]:
# Remove the 'GitHubToken' parameter from the template
del template['Parameters']['GitHubToken']

# Write the corrected template to a JSON file
with open('template.json', 'w') as f:
  json.dump(template, f, indent=2)

# Convert the corrected template to YAML
template_yaml = yaml.dump(template)

# Write the corrected template to a YAML file
with open('template.yaml', 'w') as f:
  f.write(template_yaml)

In [ ]:
import boto3

# Create a CloudFormation client
cf = boto3.client('cloudformation')

# Define the CloudFormation parameters
parameters = [
  {
    'ParameterKey': 'GitHubToken',
    'ParameterValue': '/api/github'
  }
]

# Deploy the CloudFormation template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=template_yaml,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  # Get the status of the stack
  stack = cf.describe_stacks(StackName=stack_id)['Stacks'][0]
  status = stack['StackStatus']

  # Print the status
  print(f'Stack status: {status}')

  # Break the loop if the stack is no longer in progress
  if 'IN_PROGRESS' not in status:
    break

  # Wait for a while before checking the status again
  time.sleep(5)

# Validate success
if status == 'CREATE_COMPLETE':
  print('The stack was successfully deployed.')
else:
  print('The stack deployment failed.')

In [ ]:
import boto3

# Create a CloudFormation client
cf = boto3.client('cloudformation', region_name=os.getenv('AWS_REGION'))

# Define the CloudFormation parameters
parameters = [
  {
    'ParameterKey': 'GitHubToken',
    'ParameterValue': '/api/github'
  }
]

# Deploy the CloudFormation template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=template_yaml,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  # Get the status of the stack
  stack = cf.describe_stacks(StackName=stack_id)['Stacks'][0]
  status = stack['StackStatus']

  # Print the status
  print(f'Stack status: {status}')

  # Break the loop if the stack is no longer in progress
  if 'IN_PROGRESS' not in status:
    break

  # Wait for a while before checking the status again
  time.sleep(5)

# Validate success
if status == 'CREATE_COMPLETE':
  print('The stack was successfully deployed.')
else:
  print('The stack deployment failed.')

In [ ]:
# Define the CloudFormation parameters
parameters = [
  {
    'ParameterKey': 'GitHubToken',
    'ParameterValue': '/api/github'
  }
]

# Deploy the CloudFormation template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=template_yaml,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  # Get the status of the stack
  stack = cf.describe_stacks(StackName=stack_id)['Stacks'][0]
  status = stack['StackStatus']

  # Print the status
  print(f'Stack status: {status}')

  # Break the loop if the stack is no longer in progress
  if 'IN_PROGRESS' not in status:
    break

  # Wait for a while before checking the status again
  time.sleep(5)

# Validate success
if status == 'CREATE_COMPLETE':
  print('The stack was successfully deployed.')
else:
  print('The stack deployment failed.')

In [ ]:
# Define the CloudFormation parameters
parameters = [
  {
    'ParameterKey': 'GitHubToken',
    'ParameterValue': '/api/github'
  }
]

# Deploy the CloudFormation template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=template_yaml,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  # Get the status of the stack
  stack = cf.describe_stacks(StackName=stack_id)['Stacks'][0]
  status = stack['StackStatus']

  # Print the status
  print(f'Stack status: {status}')

  # Break the loop if the stack is no longer in progress
  if 'IN_PROGRESS' not in status:
    break

  # Wait for a while before checking the status again
  time.sleep(5)

# Validate success
if status == 'CREATE_COMPLETE':
  print('The stack was successfully deployed.')
else:
  print('The stack deployment failed.')

In [ ]:
# Define the parameters for the CloudFormation template
template_parameters = {
  'Parameters': {
    'GitHubBranch': {
      'Type': 'String',
      'Description': 'The branch of the GitHub repository to use.'
    },
    'GitHubOwner': {
      'Type': 'String',
      'Description': 'The owner of the GitHub repository.'
    },
    'GitHubRepo': {
      'Type': 'String',
      'Description': 'The name of the GitHub repository.'
    },
    'GitHubToken': {
      'Type': 'AWS::SSM::Parameter::Value<String>',
      'Description': 'The GitHub token stored in the SSM Parameter Store.',
      'Default': '/api/github'
    }
  }
}

# Update the CloudFormation template with the parameters
template_yaml = yaml.safe_load(template_yaml)
template_yaml.update(template_parameters)

# Convert the template back to a YAML string
template_yaml = yaml.safe_dump(template_yaml)

In [ ]:
# Define the CloudFormation parameters
parameters = [
  {
    'ParameterKey': 'GitHubBranch',
    'ParameterValue': 'main'
  },
  {
    'ParameterKey': 'GitHubOwner',
    'ParameterValue': 'matthewhand'
  },
  {
    'ParameterKey': 'GitHubRepo',
    'ParameterValue': 'github-backup'
  },
  {
    'ParameterKey': 'GitHubToken',
    'ParameterValue': '/api/github'
  }
]

# Deploy the CloudFormation template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=template_yaml,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  # Get the status of the stack
  stack = cf.describe_stacks(StackName=stack_id)['Stacks'][0]
  status = stack['StackStatus']

  # Print the status
  print(f'Stack status: {status}')

  # Break the loop if the stack is no longer in progress
  if 'IN_PROGRESS' not in status:
    break

  # Wait for a while before checking the status again
  time.sleep(5)

# Validate success
if status == 'CREATE_COMPLETE':
  print('The stack was successfully deployed.')
else:
  print('The stack deployment failed.')

In [ ]:
# Remove the GitHubToken parameter from the CloudFormation template
template_yaml = yaml.safe_load(template_yaml)
del template_yaml['Parameters']['GitHubToken']

# Convert the template back to a YAML string
template_yaml = yaml.safe_dump(template_yaml)

# Define the CloudFormation parameters
parameters = [
  {
    'ParameterKey': 'GitHubBranch',
    'ParameterValue': 'main'
  },
  {
    'ParameterKey': 'GitHubOwner',
    'ParameterValue': 'matthewhand'
  },
  {
    'ParameterKey': 'GitHubRepo',
    'ParameterValue': 'github-backup'
  }
]

# Deploy the CloudFormation template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=template_yaml,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  # Get the status of the stack
  stack = cf.describe_stacks(StackName=stack_id)['Stacks'][0]
  status = stack['StackStatus']

  # Print the status
  print(f'Stack status: {status}')

  # Break the loop if the stack is no longer in progress
  if 'IN_PROGRESS' not in status:
    break

  # Wait for a while before checking the status again
  time.sleep(5)

# Validate success
if status == 'CREATE_COMPLETE':
  print('The stack was successfully deployed.')
else:
  print('The stack deployment failed.')

In [ ]:
# Add the GitHubToken parameter back to the CloudFormation template
template_yaml = yaml.safe_load(template_yaml)
template_yaml['Parameters']['GitHubToken'] = {
  'Default': '/api/github',
  'Description': 'The GitHub personal access token stored as a secure string in SSM Parameter Store.',
  'Type': 'AWS::SSM::Parameter::Value<String>'
}

# Convert the template back to a YAML string
template_yaml = yaml.safe_dump(template_yaml)

# Define the CloudFormation parameters
parameters = [
  {
    'ParameterKey': 'GitHubBranch',
    'ParameterValue': 'main'
  },
  {
    'ParameterKey': 'GitHubOwner',
    'ParameterValue': 'matthewhand'
  },
  {
    'ParameterKey': 'GitHubRepo',
    'ParameterValue': 'github-backup'
  },
  {
    'ParameterKey': 'GitHubToken',
    'ParameterValue': '/api/github'
  }
]

# Deploy the CloudFormation template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=template_yaml,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  # Get the status of the stack
  stack = cf.describe_stacks(StackName=stack_id)['Stacks'][0]
  status = stack['StackStatus']

  # Print the status
  print(f'Stack status: {status}')

  # Break the loop if the stack is no longer in progress
  if 'IN_PROGRESS' not in status:
    break

  # Wait for a while before checking the status again
  time.sleep(5)

# Validate success
if status == 'CREATE_COMPLETE':
  print('The stack was successfully deployed.')
else:
  print('The stack deployment failed.')

In [ ]:
# Add the GitHubToken parameter back to the CloudFormation template
template_yaml = yaml.safe_load(template_yaml)
template_yaml['Parameters']['GitHubToken'] = {
  'Default': '/api/github',
  'Description': 'The GitHub personal access token stored as a secure string in SSM Parameter Store.',
  'Type': 'AWS::SSM::Parameter::Value<String>'
}

# Convert the template back to a YAML string
template_yaml = yaml.safe_dump(template_yaml)

# Define the CloudFormation parameters
parameters = [
  {
    'ParameterKey': 'GitHubBranch',
    'ParameterValue': 'main'
  },
  {
    'ParameterKey': 'GitHubOwner',
    'ParameterValue': 'matthewhand'
  },
  {
    'ParameterKey': 'GitHubRepo',
    'ParameterValue': 'github-backup'
  },
  {
    'ParameterKey': 'GitHubToken',
    'ParameterValue': '/api/github'
  }
]

# Deploy the CloudFormation template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=template_yaml,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  # Get the status of the stack
  stack = cf.describe_stacks(StackName=stack_id)['Stacks'][0]
  status = stack['StackStatus']

  # Print the status
  print(f'Stack status: {status}')

  # Break the loop if the stack is no longer in progress
  if 'IN_PROGRESS' not in status:
    break

  # Wait for a while before checking the status again
  time.sleep(5)

# Validate success
if status == 'CREATE_COMPLETE':
  print('The stack was successfully deployed.')
else:
  print('The stack deployment failed.')

In [ ]:
# Add the GitHubToken parameter back to the CloudFormation template
template_yaml = yaml.safe_load(template_yaml)
template_yaml['Parameters']['GitHubToken'] = {
  'Default': '{{resolve:ssm-secure:/api/github}}',
  'Description': 'The GitHub personal access token stored as a secure string in SSM Parameter Store.',
  'Type': 'String'
}

# Convert the template back to a YAML string
template_yaml = yaml.safe_dump(template_yaml)

# Define the CloudFormation parameters
parameters = [
  {
    'ParameterKey': 'GitHubBranch',
    'ParameterValue': 'main'
  },
  {
    'ParameterKey': 'GitHubOwner',
    'ParameterValue': 'matthewhand'
  },
  {
    'ParameterKey': 'GitHubRepo',
    'ParameterValue': 'github-backup'
  },
  {
    'ParameterKey': 'GitHubToken',
    'ParameterValue': '{{resolve:ssm-secure:/api/github}}'
  }
]

# Deploy the CloudFormation template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=template_yaml,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  # Get the status of the stack
  stack = cf.describe_stacks(StackName=stack_id)['Stacks'][0]
  status = stack['StackStatus']

  # Print the status
  print(f'Stack status: {status}')

  # Break the loop if the stack is no longer in progress
  if 'IN_PROGRESS' not in status:
    break

  # Wait for a while before checking the status again
  time.sleep(5)

# Validate success
if status == 'CREATE_COMPLETE':
  print('The stack was successfully deployed.')
else:
  print('The stack deployment failed.')

In [ ]:
# Remove the GitHubToken parameter from the CloudFormation template
template_yaml = yaml.safe_load(template_yaml)
del template_yaml['Parameters']['GitHubToken']

# Convert the template back to a YAML string
template_yaml = yaml.safe_dump(template_yaml)

# Define the CloudFormation parameters
parameters = [
  {
    'ParameterKey': 'GitHubBranch',
    'ParameterValue': 'main'
  },
  {
    'ParameterKey': 'GitHubOwner',
    'ParameterValue': 'matthewhand'
  },
  {
    'ParameterKey': 'GitHubRepo',
    'ParameterValue': 'github-backup'
  }
]

# Deploy the CloudFormation template
response = cf.create_stack(
  StackName='GithubBackupPipeline',
  TemplateBody=template_yaml,
  Parameters=parameters,
  Capabilities=['CAPABILITY_NAMED_IAM']
)

# Get the ID of the stack
stack_id = response['StackId']

# Monitor the deployment
while True:
  # Get the status of the stack
  stack = cf.describe_stacks(StackName=stack_id)['Stacks'][0]
  status = stack['StackStatus']

  # Print the status
  print(f'Stack status: {status}')

  # Break the loop if the stack is no longer in progress
  if 'IN_PROGRESS' not in status:
    break

  # Wait for a while before checking the status again
  time.sleep(5)

# Validate success
if status == 'CREATE_COMPLETE':
  print('The stack was successfully deployed.')
else:
  print('The stack deployment failed.')